# End-to-End Pipeline Debug

This notebook debugs the end-to-end pipeline using the actual EndToEndPipeline class.
Tests complete pipeline workflow using real pipeline methods.

In [ ]:
# Import pipeline modules
import sys
sys.path.append('/home/kosaraju/mgpt-serve/mgpt_eval')

from models.config_models import PipelineConfig
from pipelines.end_to_end_pipeline import EndToEndPipeline
from pipelines.embedding_pipeline import EmbeddingPipeline
from pipelines.classification_pipeline import ClassificationPipeline
from pipelines.evaluation_pipeline import EvaluationPipeline
import pandas as pd
import os

In [ ]:
# Load configuration
config_path = "/home/kosaraju/mgpt-serve/mgpt_eval/configs/pipeline_config.yaml"
config = PipelineConfig.from_yaml(config_path)
print(f"Config loaded: {config.job.job_name}")
print(f"Output directory: {config.job.output_base_dir}")

In [ ]:
# Initialize end-to-end pipeline
try:
    e2e_pipeline = EndToEndPipeline(config)
    print(f"End-to-end pipeline initialized")
    print(f"Pipeline methods: {[m for m in dir(e2e_pipeline) if not m.startswith('_') and callable(getattr(e2e_pipeline, m))]}")
except Exception as e:
    print(f"End-to-end pipeline initialization failed: {e}")
    e2e_pipeline = None

In [ ]:
# Create test dataset
test_data = pd.DataFrame({
    'mcid': [f'E2E_{i:03d}' for i in range(30)],
    'claims': [
        'N6320 G0378 |eoc| Z91048 M1710',
        'E119 A1234 |eoc| B5678 C9012',
        'Z03818 D3456 |eoc| F7890 G1234',
        'H5678 I9012 |eoc| J1234 K5678',
        'L9012 M3456 |eoc| N6320 O7890',
        'P1234 Q5678 |eoc| E119 R9012',
        'S3456 T7890 |eoc| U1234 V5678',
        'W9012 X3456 |eoc| Z03818 Y7890',
        'Z1234 A5678 |eoc| B9012 C3456',
        'D7890 E1234 |eoc| N6320 F5678'
    ] * 3,
    'label': [1, 1, 1, 0, 1, 1, 0, 1, 0, 1] * 3
})

print(f"Test data created: {len(test_data)} samples")
print(f"Labels distribution: {test_data['label'].value_counts().to_dict()}")
print(test_data.head())

In [ ]:
# Test end-to-end pipeline run
if e2e_pipeline:
    print("Testing end-to-end pipeline run...")
    
    try:
        # Use the main run method
        results = e2e_pipeline.run(test_data)
        print(f"✓ End-to-end pipeline completed")
        print(f"  Results type: {type(results)}")
        print(f"  Results keys: {list(results.keys()) if isinstance(results, dict) else 'Not a dict'}")
        
        if isinstance(results, dict):
            for key, value in results.items():
                print(f"    {key}: {type(value)}")
                if hasattr(value, 'shape'):
                    print(f"      Shape: {value.shape}")
                elif isinstance(value, (list, dict)):
                    print(f"      Length: {len(value)}")
                    
    except Exception as e:
        print(f"✗ End-to-end pipeline failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("No end-to-end pipeline available for testing")

In [ ]:
# Test individual pipeline components
print("Testing individual pipeline components...")

# Test embedding pipeline
print("\n1. Testing Embedding Pipeline:")
try:
    embedding_pipeline = EmbeddingPipeline(config)
    # Use smaller subset for testing
    small_data = test_data.head(5)
    embeddings = embedding_pipeline.run(small_data)
    print(f"  ✓ Embedding pipeline completed")
    print(f"    Result type: {type(embeddings)}")
except Exception as e:
    print(f"  ✗ Embedding pipeline failed: {e}")

# Test classification pipeline
print("\n2. Testing Classification Pipeline:")
try:
    classification_pipeline = ClassificationPipeline(config)
    # Split data for classification
    split_idx = int(len(test_data) * 0.8)
    train_data = test_data[:split_idx]
    test_data_split = test_data[split_idx:]
    
    results = classification_pipeline.run(train_data, test_data_split)
    print(f"  ✓ Classification pipeline completed")
    print(f"    Result type: {type(results)}")
except Exception as e:
    print(f"  ✗ Classification pipeline failed: {e}")

# Test evaluation pipeline
print("\n3. Testing Evaluation Pipeline:")
try:
    evaluation_pipeline = EvaluationPipeline(config)
    eval_results = evaluation_pipeline.run(test_data.head(5))
    print(f"  ✓ Evaluation pipeline completed")
    print(f"    Result type: {type(eval_results)}")
except Exception as e:
    print(f"  ✗ Evaluation pipeline failed: {e}")

In [ ]:
# Test pipeline with different configurations
print("Testing with different configurations...")

config_files = [
    "/home/kosaraju/mgpt-serve/mgpt_eval/configs/examples/config_embeddings_only.yaml",
    "/home/kosaraju/mgpt-serve/mgpt_eval/configs/examples/config_evaluation_only.yaml",
    "/home/kosaraju/mgpt-serve/mgpt_eval/configs/examples/config_training_from_embeddings.yaml"
]

for config_file in config_files:
    if os.path.exists(config_file):
        print(f"\nTesting with {os.path.basename(config_file)}:")
        try:
            test_config = PipelineConfig.from_yaml(config_file)
            print(f"  Config loaded: {test_config.job.job_name}")
            
            # Try to initialize appropriate pipeline
            if 'embeddings_only' in config_file:
                pipeline = EmbeddingPipeline(test_config)
                result = pipeline.run(test_data.head(3))
                print(f"  ✓ Embeddings-only pipeline completed")
                
            elif 'evaluation_only' in config_file:
                pipeline = EvaluationPipeline(test_config)
                result = pipeline.run(test_data.head(3))
                print(f"  ✓ Evaluation-only pipeline completed")
                
            elif 'training_from_embeddings' in config_file:
                pipeline = ClassificationPipeline(test_config)
                train_subset = test_data.head(15)
                test_subset = test_data.tail(5)
                result = pipeline.run(train_subset, test_subset)
                print(f"  ✓ Training-from-embeddings pipeline completed")
                
        except Exception as e:
            print(f"  ✗ Pipeline failed: {e}")
    else:
        print(f"\nConfig file not found: {config_file}")

In [ ]:
# Debug pipeline data flow
print("=== Pipeline Data Flow Debug ===")

if e2e_pipeline:
    # Check pipeline attributes
    print(f"Pipeline config: {hasattr(e2e_pipeline, 'config')}")
    print(f"Pipeline logger: {hasattr(e2e_pipeline, 'logger')}")
    print(f"Pipeline components: {hasattr(e2e_pipeline, 'components')}")
    
    # Check for sub-pipelines
    sub_pipelines = ['embedding_pipeline', 'classification_pipeline', 'evaluation_pipeline']
    for sub_pipeline_name in sub_pipelines:
        if hasattr(e2e_pipeline, sub_pipeline_name):
            sub_pipeline = getattr(e2e_pipeline, sub_pipeline_name)
            print(f"\n{sub_pipeline_name}: {type(sub_pipeline)}")
            if hasattr(sub_pipeline, 'run'):
                print(f"  Has run method: ✓")
            else:
                print(f"  Has run method: ✗")
        else:
            print(f"\n{sub_pipeline_name}: Not found")
    
    # Check all methods and attributes
    all_attrs = [attr for attr in dir(e2e_pipeline) if not attr.startswith('_')]
    methods = [attr for attr in all_attrs if callable(getattr(e2e_pipeline, attr))]
    properties = [attr for attr in all_attrs if not callable(getattr(e2e_pipeline, attr))]
    
    print(f"\nMethods: {methods}")
    print(f"Properties: {properties}")
else:
    print("No end-to-end pipeline available for debugging")

In [ ]:
# Test pipeline step-by-step execution
print("Testing step-by-step pipeline execution...")

if e2e_pipeline and hasattr(e2e_pipeline, 'run_step_by_step'):
    try:
        step_results = e2e_pipeline.run_step_by_step(test_data.head(5))
        print(f"✓ Step-by-step execution completed")
        for step, result in step_results.items():
            print(f"  Step {step}: {type(result)}")
    except Exception as e:
        print(f"✗ Step-by-step execution failed: {e}")
else:
    print("Step-by-step execution not available")
    
    # Try manual step-by-step
    print("\nTrying manual step-by-step execution...")
    small_data = test_data.head(5)
    
    # Step 1: Embeddings
    try:
        embedding_pipeline = EmbeddingPipeline(config)
        step1_result = embedding_pipeline.run(small_data)
        print(f"Step 1 (Embeddings): ✓ {type(step1_result)}")
    except Exception as e:
        print(f"Step 1 (Embeddings): ✗ {e}")
        step1_result = None
    
    # Step 2: Classification (if embeddings succeeded)
    if step1_result is not None:
        try:
            classification_pipeline = ClassificationPipeline(config)
            # Use embeddings from step 1 if available
            step2_result = classification_pipeline.run(small_data[:3], small_data[3:])
            print(f"Step 2 (Classification): ✓ {type(step2_result)}")
        except Exception as e:
            print(f"Step 2 (Classification): ✗ {e}")
    
    # Step 3: Evaluation
    try:
        evaluation_pipeline = EvaluationPipeline(config)
        step3_result = evaluation_pipeline.run(small_data)
        print(f"Step 3 (Evaluation): ✓ {type(step3_result)}")
    except Exception as e:
        print(f"Step 3 (Evaluation): ✗ {e}")

In [ ]:
# Test pipeline output handling
print("Testing pipeline output handling...")

# Check if output directory exists
output_dir = config.job.output_base_dir
print(f"Output directory: {output_dir}")
print(f"Directory exists: {os.path.exists(output_dir)}")

if not os.path.exists(output_dir):
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"✓ Created output directory: {output_dir}")
    except Exception as e:
        print(f"✗ Failed to create output directory: {e}")

# Test if pipeline can save results
if e2e_pipeline and hasattr(e2e_pipeline, 'save_results'):
    try:
        mock_results = {'test': 'data', 'embeddings': [[1, 2, 3]], 'predictions': [1, 0, 1]}
        e2e_pipeline.save_results(mock_results)
        print(f"✓ Results saving tested")
    except Exception as e:
        print(f"✗ Results saving failed: {e}")
else:
    print("No save_results method available")